In [1]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd

In [2]:
ee.Initialize()

# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')

# AREA OF INTEREST
region_L1 = ee.Geometry.Polygon([[[-30, -40], [65, -30], [65, 40], [-30, 40]]])
region_json= [[-30, -40], [65, -30], [65, 40], [-30, 40]]

In [3]:
subregions = {
  'na': 'Northern Africa',
  'ma': 'Middle Africa',
  'wa': 'Western Africa', 
  'ea': 'Eastern Africa',
  'sa': 'Southern Africa', 
  'ws': 'Western Asia'
}

# <span style="color:red">INPUTS</span> 

In [49]:
subregion = 'na'
year = 2010
name_asset = 'T_Annual'

In [50]:
selected_subregion = subregions[subregion]
filtered = countries_gaul.filter(ee.Filter.eq('subregion', selected_subregion))
country_number = filtered.size().getInfo()
country_number

13

In [51]:
radice = 'projects/fao-wapor/'
raster_name = radice + name_asset + "/" + name_asset + "-" + str(year)
print(raster_name)
prefix = raster_name.split("/")[2]

projects/fao-wapor/T_Annual/T_Annual-2010


In [52]:
raster_stats = ee.Image(raster_name).toFloat();
scale_calc = ee.Image(raster_stats).projection().nominalScale().getInfo();
#print('Image properties: ', raster_stats.getInfo());
print('Image scale: ', scale_calc);

('Image scale: ', 248.48100621908284)


In [53]:
def vizualize_image(raster,regione_plot,scale):    
    visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region": regione_plot,
             "scale" : scale_calc}
    
    url_raster_stat= raster.getThumbUrl(visparams)
    return url_raster_stat

In [54]:
#%matplotlib inline  
#Image(url=vizualize_image(raster_stats,region_json,scale_calc))

#### Means

In [55]:
means = raster_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = scale_calc,
)

#### MinMax Sum combined

In [56]:
reducers = ee.Reducer.minMax().combine(
    reducer2 = ee.Reducer.sum(),
    sharedInputs = True
);

minMaxStds = raster_stats.reduceRegions(
            collection = filtered,
            reducer = reducers,
            scale = scale_calc,
);

## Export

In [57]:
features_m = means.getInfo()['features']
features_mms = minMaxStds.getInfo()['features']

In [58]:
df_m = pd.DataFrame(data=features_m[1:], columns = features_m[0])
df_mms = pd.DataFrame(data=features_mms[1:], columns = features_mms[0])

In [59]:
serie_m = df_m['properties'].apply(pd.Series)
serie_mms = df_mms['properties'].apply(pd.Series)

In [60]:
df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])

In [61]:
df_mms = pd.DataFrame(serie_mms[['fid', 'min', 'max','sum']])

In [62]:
# caller.set_index('key').join(other.set_index('key'))
df_stats = df_m.set_index('fid').join(df_mms.set_index('fid'), lsuffix='_df_m', rsuffix='_df_mms')

In [63]:
df_stats

,gaul_code,iso3,mean,name,region,subregion,min,max,sum
fid,,,,,,,,,
6.0,6.0,SDN,139.934674,Sudan,Africa,Northern Africa,0.000000,4232.866211,4.348628e+09
270.0,40760.0,,0.414415,Hala'ib triangle,Africa,Northern Africa,0.000000,1262.863037,1.285611e+05
272.0,40765.0,EGY,48.013704,Egypt,Africa,Northern Africa,0.000000,3886.138916,8.567724e+08
142.0,145.0,LBY,7.462003,Libya,Africa,Northern Africa,0.000000,3148.794922,2.210462e+08
274.0,61013.0,,520.132087,Ilemi triangle,Africa,Northern Africa,83.279999,2010.340942,2.698185e+07
276.0,74582.0,,609.064280,Kafia Kenji,Africa,Northern Africa,330.119995,1479.376953,1.156662e+08
277.0,74583.0,,568.281284,Kaka,Africa,Northern Africa,0.000000,2602.117920,5.434474e+06
166.0,169.0,MAR,115.978170,Morocco,Africa,Northern Africa,0.000000,1876.813965,9.184503e+08
74.0,74.0,SSD,762.440065,South Sudan,Africa,Northern Africa,0.000000,3028.899902,7.888658e+09


In [64]:
file_name = name_asset + "_" + str(year) + "_" + subregion # + prefix.lower()
file_name

'T_Annual_2010_na'

In [65]:
df_stats.to_csv('stats/' + file_name + '.csv')